In [1]:
___Author___='LumberJack Jyss'

# LumberJack_Haute_Tension_Forex_01

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.layers.recurrent import LSTM
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
sns.set(palette='bright',style='whitegrid')
import numpy as np
import pandas as pd
import quandl
import pandas_datareader as web
from statsmodels.tsa.stattools import coint
import statsmodels
from statsmodels.tsa.stattools import adfuller
import itertools
import scipy
from scipy import stats
import time
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
data = pd.read_csv('raw_data_solid.csv',delimiter=';')

### On choisit 'USDJPY Close' comme target, et on le shift de une heure
En réalité, il manque des minutes. Donc on shift arbitrairement de 60

Il faudra vraiment des données solides

In [4]:
dataset = data
dataset['USDJPY Close'] = dataset['USDJPY Close'].shift(-60)
dataset.tail()

,Date,AUDUSD Close,EURUSD Close,GBPUSD Close,NZDUSD Close,USDCAD Close,USDCHF Close,USDJPY Close
452916,2019-03-22 15:54:00,0.70792,1.13002,1.32091,0.68763,1.34285,0.99342,NaN
452917,2019-03-22 15:55:00,0.70796,1.13001,1.32106,0.68733,1.34259,0.99343,NaN
452918,2019-03-22 15:56:00,0.70796,1.13000,1.32110,0.68726,1.34250,0.99343,NaN
452919,2019-03-22 15:57:00,0.70796,1.13009,1.32083,0.68742,1.34280,0.99336,NaN
452920,2019-03-22 15:58:00,0.70786,1.13007,1.32088,0.68729,1.34278,0.99312,NaN


In [5]:
dataset.set_index('Date', inplace=True,drop=True)
dataset.head()

,AUDUSD Close,EURUSD Close,GBPUSD Close,NZDUSD Close,USDCAD Close,USDCHF Close,USDJPY Close
Date,,,,,,,
2018-01-01 17:00:00,0.78040,1.20100,1.34985,0.71027,1.25425,0.97472,112.660
2018-01-01 17:01:00,0.78005,1.20030,1.34985,0.71027,1.25425,0.97472,112.656
2018-01-01 17:02:00,0.78038,1.20043,1.35019,0.71027,1.25425,0.97429,112.648
2018-01-01 17:04:00,0.78037,1.20048,1.35019,0.71027,1.25440,0.97475,112.660
2018-01-01 17:05:00,0.77968,1.20048,1.35012,0.71027,1.25441,0.97475,112.650


On retire les 60 dernières lignes qui ne nous servent plus et on sépare les X des y

In [6]:
X = dataset.iloc[:-60:, :6]#.to_frame()#.dropna()
y = dataset.iloc[:-60:, 6].to_frame()#.dropna()

In [7]:
X.isnull().sum().sum()

0

In [8]:
y.isnull().sum().sum()

0

In [9]:
X.head()

,AUDUSD Close,EURUSD Close,GBPUSD Close,NZDUSD Close,USDCAD Close,USDCHF Close
Date,,,,,,
2018-01-01 17:00:00,0.78040,1.20100,1.34985,0.71027,1.25425,0.97472
2018-01-01 17:01:00,0.78005,1.20030,1.34985,0.71027,1.25425,0.97472
2018-01-01 17:02:00,0.78038,1.20043,1.35019,0.71027,1.25425,0.97429
2018-01-01 17:04:00,0.78037,1.20048,1.35019,0.71027,1.25440,0.97475
2018-01-01 17:05:00,0.77968,1.20048,1.35012,0.71027,1.25441,0.97475


In [10]:
y.head()

,USDJPY Close
Date,
2018-01-01 17:00:00,112.660
2018-01-01 17:01:00,112.656
2018-01-01 17:02:00,112.648
2018-01-01 17:04:00,112.660
2018-01-01 17:05:00,112.650


In [11]:
X.iloc[2,0]

0.7803800000000001

In [12]:
len(X)

452861

In [13]:
len(y)

452861

### DEEP LEARNIG

In [14]:
print ('\nInfos X : \n')
print (X.info(),'\n')


Infos X : 

<class 'pandas.core.frame.DataFrame'>
Index: 452861 entries, 2018-01-01 17:00:00 to 2019-03-22 14:58:00
Data columns (total 6 columns):
AUDUSD Close    452861 non-null float64
EURUSD Close    452861 non-null float64
GBPUSD Close    452861 non-null float64
NZDUSD Close    452861 non-null float64
USDCAD Close    452861 non-null float64
USDCHF Close    452861 non-null float64
dtypes: float64(6)
memory usage: 44.2+ MB
None 



In [15]:
print ('\nInfos y : \n')
print(y.info(),'\n')


Infos y : 

<class 'pandas.core.frame.DataFrame'>
Index: 452861 entries, 2018-01-01 17:00:00 to 2019-03-22 14:58:00
Data columns (total 1 columns):
USDJPY Close    452861 non-null float64
dtypes: float64(1)
memory usage: 6.9+ MB
None 



In [16]:
# Dimensions of dataset sur 2 colonnes
#X.dropna()
#y.dropna()
n = X.shape[0]
p = y.shape[0]
print('La valeur de n (shape de X) est de : ',n)
print('La valeur de p (shape de y) est de : ',p)

La valeur de n (shape de X) est de :  452861
La valeur de p (shape de y) est de :  452861


In [17]:
X = X.values
y = y.values

In [18]:
X.shape

(452861, 6)

In [19]:
y.shape

(452861, 1)

In [20]:
# Training and test data
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end
test_end = n
X_train = X[np.arange(train_start, train_end), :]
X_test = X[np.arange(test_start, test_end), :]
y_train = y[np.arange(train_start, train_end), :]
y_test = y[np.arange(test_start, test_end), :]

In [21]:
# Scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

In [22]:
# Initialize the graph
graph = tf.Session()

In [23]:
# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, 6]) # correspond au shape de X 
Y = tf.placeholder(dtype=tf.float32, shape=[None,1]) # correspond au shape de y

In [24]:
# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [25]:
# Model architecture parameters
n_stocks = 6 # Nb of columns
n_neurons_1 = 100
n_neurons_2 = 100
n_neurons_3 = 100
n_neurons_4 = 100
n_neurons_5 = 100
n_target = 1

# Layer 1: Variables for hidden weights and biases
W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
# Layer 2: Variables for hidden weights and biases
#W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
#bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
# Layer 3: Variables for hidden weights and biases
#W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
#bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
# Layer 4: Variables for hidden weights and biases
#W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
#bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))
# Layer 4: Variables for hidden weights and biases
W_hidden_5 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_5]))
bias_hidden_5 = tf.Variable(bias_initializer([n_neurons_5]))

# Output layer: Variables for output weights and biases
W_out = tf.Variable(weight_initializer([n_neurons_5, n_target]))
bias_out = tf.Variable(bias_initializer([n_target]))

In [29]:
# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
#hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
#hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
#hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))
hidden_5 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_5), bias_hidden_5))


# Output layer (must be transposed)
out = tf.transpose(tf.add(tf.matmul(hidden_5, W_out), bias_out))

In [30]:
# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

In [31]:
# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

In [ ]:
# Make Session
net = tf.Session()
# Run initializer
net.run(tf.global_variables_initializer())

# Setup interactive plot
#plt.ion()
#fig = plt.figure(figsize=(16,6))
#ax1 = fig.add_subplot(111)
#line1, = ax1.plot(y_test)
#line2, = ax1.plot(y_test*0.5)
#plt.legend()
#plt.show()

# Number of epochs and batch size
epochs = 1
batch_size = 100000

for e in range(epochs):

    # Shuffle training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    X_train = X_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    # Minibatch training
    for i in range(0, len(y_train) // batch_size):
        start = i * batch_size
        batch_x = X_train[start:start + batch_size]
        batch_y = y_train[start:start + batch_size]
        # Run optimizer with batch
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        # Show progress
        if np.mod(i, 5) == 0:
            # Prediction
            pred = net.run(out, feed_dict={X: X_test})
            line2.set_ydata(pred)
            plt.title('Epoch ' + str(e) + ', Batch ' + str(i))
            file_name = 'img/epoch_' + str(e) + '_batch_' + str(i) + '.jpg'
           # plt.savefig(file_name)
            plt.pause(0.01)
# Print final MSE after Training

In [ ]:
mse_final = net.run(mse, feed_dict={X: X_test, Y: y_test})

In [ ]:
print(mse_final)